In [ ]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Definisi simbol
x, y, k, q1, q2, a, b = sp.symbols('x y k q1 q2 a b')

# Fungsi potensial listrik
V = k * (q1 / sp.sqrt(x**2 + y**2) + q2 / sp.sqrt((x - a)**2 + (y - b)**2))

# Gradien (∇V)
grad_V = [sp.diff(V, x), sp.diff(V, y)]
grad_V
# Matriks Jacobian
J = sp.Matrix([[sp.diff(grad_V[0], x), sp.diff(grad_V[0], y)],
               [sp.diff(grad_V[1], x), sp.diff(grad_V[1], y)]])

# Fungsi untuk Newton-Raphson iteratif
def newton_raphson_2d(f_grad, J_mat, x0, y0, params, tol=1e-6, max_iter=50):
    Xn = np.array([x0, y0], dtype=float)
    
    for _ in range(max_iter):
        grad_eval = np.array([f.subs(params).evalf() for f in f_grad], dtype=float)
        J_eval = np.array(J_mat.subs(params).evalf(), dtype=float)
        
        if np.linalg.norm(grad_eval) < tol:
            break

        try:
            delta = np.linalg.solve(J_eval, -grad_eval)
        except np.linalg.LinAlgError:
            print("Jacobian singular, metode gagal.")
            return None
        
        Xn += delta

    return Xn
H = J  # Hessian sama dengan Jacobian dari gradien
det_H = H.det()
saddle_condition = det_H < 0
saddle_condition
# Parameter numerik
param_values = {k: 1, q1: 1, q2: -1, a: 2, b: 2}
x_vals = np.linspace(-3, 3, 100)
y_vals = np.linspace(-3, 3, 100)
X, Y = np.meshgrid(x_vals, y_vals)

# Evaluasi fungsi
V_func = sp.lambdify((x, y), V.subs(param_values), 'numpy')
Z = V_func(X, Y)

# Plot 3D
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, Z, cmap='viridis')

# Plot titik saddle point jika ditemukan
saddle_point = newton_raphson_2d(grad_V, H, 1, 1, param_values)
if saddle_point is not None:
    ax.scatter(saddle_point[0], saddle_point[1], V_func(saddle_point[0], saddle_point[1]), 
               color='r', s=100, label="Saddle Point")

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('V(x, y)')
ax.legend()
plt.show()
